In [0]:
# -----------------------------------#
# 1o - Ingestão de dados             #
# 2o - Criação do modelo             #
# 3o - Treinamento do modelo         #
# 4o - Teste do modelo               #
# 5o - Medir a performance do modelo #
# -----------------------------------#

# Casse = o que eu quero prever ou classificar, também é um atributo/classe
# Dimensão ou atributo = características ou colunas
# Instãncia =  observação ou linha
# Relação = conjunto de dados

# Classificação binária = classificação de sim ou não em uma única classe/coluna
# Classificação multiclasse = mais de uma classificação em uma mesma classe/coluna
# Classificação multilabel = classificação por meio de mais de uma classe/coluna

# Variáveis independentes devem estar em uma única coluna
# Variáveis dependentes = a qual se quer avaliar
# One Hot Encoding = todos os dados devem ser convertidos em números

In [0]:
# -------------------------------------------------------------------------#
# Usando modelos de regressão Linear Regression e Random Forest Regression #
# Dado um projeto de um carro, queremos prever qual a potência HP do mesmo #
# -------------------------------------------------------------------------#

In [0]:
# bibliotecas para gerar modelos de regressão
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

# biblioteca para avaliar a performance do modelo de regressão
from pyspark.ml.evaluation import RegressionEvaluator

# biblioteca para transformar dados categoricos em númericos e agrupa variáveis independentes em uma única coluna
from pyspark.ml.feature import VectorAssembler

In [0]:
# Ingestão de dados
carros_tmp = spark.read.csv('/FileStore/tables/Carros.csv', inferSchema=True, header=True, sep=";")

# Criando um subconjunto de dados
carros = carros_tmp.select("Consumo", "Cilindros", "Cilindradas", "HP")
carros.show(5)

+-------+---------+-----------+---+
Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
 21| 6| 160|110|
 21| 6| 160|110|
 228| 4| 108| 93|
 214| 6| 258|110|
 187| 8| 360|175|
+-------+---------+-----------+---+
only showing top 5 rows

In [0]:
# Criar um objeto que agrupa variáveis independentes em uma úica coluna
vector = VectorAssembler(inputCols=[("Consumo"),("Cilindros"),("Cilindradas")], outputCol="VectorColumn")

# Aplica os dados do df carros ao objeto de transformação vector
carros = vector.transform(carros)
carros.show(5)

+-------+---------+-----------+---+-----------------+
Consumo|Cilindros|Cilindradas| HP| VectorColumn|
+-------+---------+-----------+---+-----------------+
 21| 6| 160|110| [21.0,6.0,160.0]|
 21| 6| 160|110| [21.0,6.0,160.0]|
 228| 4| 108| 93|[228.0,4.0,108.0]|
 214| 6| 258|110|[214.0,6.0,258.0]|
 187| 8| 360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows

In [0]:
# Dividir a massa de dados para treino 70% e para teste 30%
carros_treino, carros_teste = carros.randomSplit([0.7,0.3])
print(carros_treino.count())
print(carros_teste.count())

21
11

In [0]:
# Cria objeto do tipo Linear Regression usando as colunas de variáveis independentes e de variável dependente
obj_lr = LinearRegression(featuresCol="VectorColumn", labelCol="HP")
# Cria o modelo aplicando o objeto LR aos dados de treino
modelo_lr = obj_lr.fit(carros_treino)

# Cria objeto do tipo Random Forest Regression usando as colunas de variáveis independentes e de variável dependente
obj_rf = RandomForestRegressor(featuresCol="VectorColumn", labelCol="HP")
# Cria o modelo aplicando o objeto RFRegression aos dados de treino
modelo_rf = obj_rf.fit(carros_treino)

In [0]:
# Realizando a previsão usando modelos criados com base nos dados de teste
previsao_lr = modelo_lr.transform(carros_teste)
previsao_lr.show()

previsao_rf = modelo_rf.transform(carros_teste)
previsao_rf.show()

+-------+---------+-----------+---+------------------+------------------+
Consumo|Cilindros|Cilindradas| HP| VectorColumn| prediction|
+-------+---------+-----------+---+------------------+------------------+
 21| 6| 160|110| [21.0,6.0,160.0]| 180.8038821638169|
 26| 4| 1203| 91| [26.0,4.0,1203.0]| 112.6920022656728|
 104| 8| 472|205| [104.0,8.0,472.0]|219.43372632349732|
 155| 8| 318|150| [155.0,8.0,318.0]|211.39795395741908|
 158| 8| 351|264| [158.0,8.0,351.0]|210.52000655579522|
 173| 8| 2758|180|[173.0,8.0,2758.0]| 184.527632315867|
 178| 6| 1676|123|[178.0,6.0,1676.0]|136.89104417400932|
 187| 8| 360|175| [187.0,8.0,360.0]| 205.0201654119956|
 214| 4| 121|109| [214.0,4.0,121.0]| 88.02557405362326|
 228| 4| 1408| 95|[228.0,4.0,1408.0]| 73.0115423864975|
 273| 4| 79| 66| [273.0,4.0,79.0]| 77.41735826259475|
+-------+---------+-----------+---+------------------+------------------+

+-------+---------+-----------+---+------------------+------------------+
Consumo|Cilindros|Cilindradas| HP| VectorColumn| prediction|
+-------+---------+-----------+---+------------------+------------------+
 21| 6| 160|110| [21.0,6.0,160.0]| 138.76125|
 26| 4| 1203| 91| [26.0,4.0,1203.0]| 150.43|
 104| 8| 472|205| [104.0,8.0,472.0]|215.26041666666669|
 155| 8| 318|150| [155.0,8.0,318.0]|176.11458333333334|
 158| 8| 351|264| [158.0,8.0,351.0]|192.32291666666669|
 173| 8| 2758|180|[173.0,8.0,2758.0]|167.47708333333333|
 178| 6| 1676|123|[178.0,6.0,1676.0]|122.39958333333334|
 187| 8| 360|175| [187.0,8.0,360.0]|182.32708333333332|
 214| 4| 121|109| [214.0,4.0,121.0]|110.45654761904761|
 228| 4| 1408| 95|[228.0,4.0,1408.0]| 82.69444805194806|
 273| 4| 79| 66| [273.0,4.0,79.0]| 94.3727813852814|
+-------+---------+-----------+---+------------------+------------------+

In [0]:
# Cria objeto para avaliação da previsão dos modelos utilizados com dados de teste
obj_avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP", metricName="rmse")

# Avalia as previsões feitas com os modelos LR e RF com base nos dados de teste
# Na métrica RMSE quanto menor o valor, mais precisa é a previsão
avaliacao_lr = obj_avaliar.evaluate(previsao_lr)
print(avaliacao_lr)

avaliacao_rf = obj_avaliar.evaluate(previsao_rf)
print(avaliacao_rf)

36.294664486402255
32.26793968268668

In [0]:
# --------------------------------------------------------------------------#
# Usando modelos de classificação binária, classe com valores True ou False #
# Dado um determinado cliente, prever se ele vai deixar de comprar ou não   #
# --------------------------------------------------------------------------#

In [0]:
# biblioteca para criar um modelos de classificação
from pyspark.ml.feature import RFormula

# biblioteca para cirar algoritmo de classificação árvore de decisão
from pyspark.ml.classification import DecisionTreeClassifier

# biblioteca para avaliar a performance do modelo de classificação binária
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
# Ingestão de dados
churn = spark.read.csv('/FileStore/tables/Churn.csv', inferSchema=True, header=True, sep=";")
churn.show()

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
 619| France|Female| 42| 2| 0| 1| 1| 1| 10134888| 1|
 608| Spain|Female| 41| 1| 8380786| 1| 0| 1| 11254258| 0|
 502| France|Female| 42| 8| 1596608| 3| 1| 0| 11393157| 1|
 699| France|Female| 39| 1| 0| 2| 0| 0| 9382663| 0|
 850| Spain|Female| 43| 2|12551082| 1| 1| 1| 790841| 0|
 645| Spain| Male| 44| 8|11375578| 2| 1| 0| 14975671| 1|
 822| France| Male| 50| 7| 0| 2| 1| 1| 100628| 0|
 376| Germany|Female| 29| 4|11504674| 4| 1| 0| 11934688| 1|
 501| France| Male| 44| 4|14205107| 2| 0| 1| 749405| 0|
 684| France| Male| 27| 2|13460388| 1| 1| 1| 7172573| 0|
 528| France| Male| 31| 6|10201672| 2| 0| 0| 8018112| 0|
 497| Spain| Male| 24| 3| 0| 2| 1| 0| 7639001| 0|
 476| France|Female| 34| 10| 0| 2| 1| 0| 2626098| 0|
 549| France|Female| 25| 5| 0| 2| 0| 0| 19085779| 0|
 635| Spain|Female| 35| 7| 0| 2| 1| 1| 6595165| 0|
 616| Germany| Male| 45| 3|14312941| 2| 0| 1| 6432726| 0|
 653| Germany| Male| 58| 1|13260288| 1| 1| 0| 509767| 1|
 549| Spain|Female| 24| 9| 0| 2| 1| 1| 1440641| 0|
 587| Spain| Male| 45| 6| 0| 1| 0| 0| 15868481| 0|
 726| France|Female| 24| 6| 0| 2| 1| 1| 5472403| 0|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
only showing top 20 rows

In [0]:
# Criando objeto R Fórmula para criar modelo de classificação
formula = RFormula(formula="Exited ~ .", featuresCol="featuresCol", labelCol="labelCol", handleInvalid="skip")

# Realizando One Hot Encoding dos dados usando o modelo criado
churn_transformado = formula.fit(churn).transform(churn).select("featuresCol", "labelCol")
churn_transformado.show(truncate=False)

+----------------------------------------------------------------+--------+
featuresCol |labelCol|
+----------------------------------------------------------------+--------+
[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7] |1.0 |
[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7] |0.0 |
[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7] |1.0 |
(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0]) |0.0 |
[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0] |0.0 |
[645.0,0.0,0.0,1.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7]|1.0 |
[822.0,1.0,0.0,1.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0] |0.0 |
[376.0,0.0,1.0,0.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7]|1.0 |
[501.0,1.0,0.0,1.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0] |0.0 |
[684.0,1.0,0.0,1.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0] |0.0 |
[528.0,1.0,0.0,1.0,31.0,6.0,1.0201672E7,2.0,0.0,0.0,8018112.0] |0.0 |
[497.0,0.0,0.0,1.0,24.0,3.0,0.0,2.0,1.0,0.0,7639001.0] |0.0 |
[476.0,1.0,0.0,0.0,34.0,10.0,0.0,2.0,1.0,0.0,2626098.0] |0.0 |
(11,[0,1,4,5,7,10],[549.0,1.0,25.0,5.0,2.0,1.9085779E7]) |0.0 |
[635.0,0.0,0.0,0.0,35.0,7.0,0.0,2.0,1.0,1.0,6595165.0] |0.0 |
[616.0,0.0,1.0,1.0,45.0,3.0,1.4312941E7,2.0,0.0,1.0,6432726.0] |0.0 |
[653.0,0.0,1.0,1.0,58.0,1.0,1.3260288E7,1.0,1.0,0.0,509767.0] |1.0 |
[549.0,0.0,0.0,0.0,24.0,9.0,0.0,2.0,1.0,1.0,1440641.0] |0.0 |
(11,[0,3,4,5,7,10],[587.0,1.0,45.0,6.0,1.0,1.5868481E7]) |0.0 |
[726.0,1.0,0.0,0.0,24.0,6.0,0.0,2.0,1.0,1.0,5472403.0] |0.0 |
+----------------------------------------------------------------+--------+
only showing top 20 rows

In [0]:
# Dividindo os dados em treino e teste
churnTreino, churnTeste = churn_transformado.randomSplit([0.7,0.3])
print(churnTreino.count())
print(churnTeste.count())

7094
2906

In [0]:
# Criando objeto para geração do modelo de classificação
objeto_dt = DecisionTreeClassifier(labelCol="labelCol", featuresCol="featuresCol")

# criando modelo de classificação
modelo_dt = objeto_dt.fit(churnTreino)

In [0]:
# Realizando previsões com o modelo criado
previsao_dt = modelo_dt.transform(churnTeste)
previsao_dt.show(truncate=False)

+--------------------------------------------------------+--------+--------------+----------------------------------------+----------+
featuresCol |labelCol|rawPrediction |probability |prediction|
+--------------------------------------------------------+--------+--------------+----------------------------------------+----------+
(11,[0,1,3,4,7,10],[624.0,1.0,1.0,37.0,2.0,1.1210455E7])|0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,3,4,7,10],[794.0,1.0,1.0,33.0,2.0,1.7812271E7])|0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[350.0,1.0,60.0,3.0,1.0,1.1379615E7])|1.0 |[38.0,243.0] |[0.13523131672597866,0.8647686832740213]|1.0 |
(11,[0,1,4,5,7,10],[418.0,1.0,39.0,2.0,2.0,904171.0]) |0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[476.0,1.0,40.0,4.0,2.0,1.8254704E7])|0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[512.0,1.0,31.0,7.0,2.0,4932607.0]) |0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[515.0,1.0,28.0,9.0,2.0,9414175.0]) |0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[523.0,1.0,73.0,7.0,2.0,1308839.0]) |1.0 |[4.0,0.0] |[1.0,0.0] |0.0 |
(11,[0,1,4,5,7,10],[530.0,1.0,29.0,5.0,2.0,1.2145121E7])|0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[537.0,1.0,38.0,10.0,1.0,5233797.0]) |1.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[545.0,1.0,55.0,5.0,1.0,1003477.0]) |1.0 |[38.0,243.0] |[0.13523131672597866,0.8647686832740213]|1.0 |
(11,[0,1,4,5,7,10],[548.0,1.0,37.0,9.0,2.0,9802958.0]) |0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[549.0,1.0,25.0,5.0,2.0,1.9085779E7])|0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[549.0,1.0,34.0,4.0,2.0,1.3946357E7])|0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[553.0,1.0,27.0,3.0,2.0,1.5980016E7])|0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[564.0,1.0,27.0,6.0,1.0,781976.0]) |0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
(11,[0,1,4,5,7,10],[567.0,1.0,47.0,2.0,1.0,1.1090043E7])|1.0 |[122.0,118.0] |[0.5083333333333333,0.49166666666666664]|0.0 |
(11,[0,1,4,5,7,10],[568.0,1.0,44.0,7.0,2.0,6237067.0]) |1.0 |[180.0,39.0] |[0.821917808219178,0.1780821917808219] |0.0 |
(11,[0,1,4,5,7,10],[569.0,1.0,44.0,4.0,2.0,1.3439478E7])|0.0 |[180.0,39.0] |[0.821917808219178,0.1780821917808219] |0.0 |
(11,[0,1,4,5,7,10],[571.0,1.0,35.0,8.0,2.0,8456913.0]) |0.0 |[4354.0,505.0]|[0.8960691500308705,0.10393084996912945]|0.0 |
+--------------------------------------------------------+--------+--------------+----------------------------------------+----------+
only showing top 20 rows

In [0]:
# Avaliar a performance do modelo usando métrica de área sobre a curva, quanto mais próximo de 1 melhor
obj_avaliar_dt = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="labelCol", metricName="areaUnderROC")
avaliar_dt = obj_avaliar_dt.evaluate(previsao_dt)
print(avaliar_dt)

0.6688715441024111

In [0]:
# --------------------------------------------------------------------------#
# Usando modelos de classificação multi-classes                             #
# Dado uma determinada planta, prever sua classificação                     #
# --------------------------------------------------------------------------#

# biblioteca para criar um modelos de classificação
from pyspark.ml.feature import RFormula
# biblioteca para cirar algoritmo de classificação árvore de decisão
from pyspark.ml.classification import NaiveBayes
# biblioteca para avaliar a performance do modelo de classificação binária
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Ingestão de dados
iris = spark.read.csv('/FileStore/tables/iris.csv', inferSchema=True, header=True, sep=",")
iris.show(truncate=False)

+-----------+----------+-----------+----------+-----------+
sepallength|sepalwidth|petallength|petalwidth|class |
+-----------+----------+-----------+----------+-----------+
5.1 |3.5 |1.4 |0.2 |Iris-setosa|
4.9 |3.0 |1.4 |0.2 |Iris-setosa|
4.7 |3.2 |1.3 |0.2 |Iris-setosa|
4.6 |3.1 |1.5 |0.2 |Iris-setosa|
5.0 |3.6 |1.4 |0.2 |Iris-setosa|
5.4 |3.9 |1.7 |0.4 |Iris-setosa|
4.6 |3.4 |1.4 |0.3 |Iris-setosa|
5.0 |3.4 |1.5 |0.2 |Iris-setosa|
4.4 |2.9 |1.4 |0.2 |Iris-setosa|
4.9 |3.1 |1.5 |0.1 |Iris-setosa|
5.4 |3.7 |1.5 |0.2 |Iris-setosa|
4.8 |3.4 |1.6 |0.2 |Iris-setosa|
4.8 |3.0 |1.4 |0.1 |Iris-setosa|
4.3 |3.0 |1.1 |0.1 |Iris-setosa|
5.8 |4.0 |1.2 |0.2 |Iris-setosa|
5.7 |4.4 |1.5 |0.4 |Iris-setosa|
5.4 |3.9 |1.3 |0.4 |Iris-setosa|
5.1 |3.5 |1.4 |0.3 |Iris-setosa|
5.7 |3.8 |1.7 |0.3 |Iris-setosa|
5.1 |3.8 |1.5 |0.3 |Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 20 rows

In [0]:
# Criando objeto R Fórmula para criar modelo de classificação
formula = RFormula(formula="class ~ .", featuresCol="featuresCol", labelCol="labelCol", handleInvalid="skip")

# Realizando One Hot Encoding dos dados usando o modelo criado
iris_transformado = formula.fit(iris).transform(iris).select("featuresCol", "labelCol")
iris_transformado.show(truncate=False)

+-----------------+--------+
featuresCol |labelCol|
+-----------------+--------+
[5.1,3.5,1.4,0.2]|0.0 |
[4.9,3.0,1.4,0.2]|0.0 |
[4.7,3.2,1.3,0.2]|0.0 |
[4.6,3.1,1.5,0.2]|0.0 |
[5.0,3.6,1.4,0.2]|0.0 |
[5.4,3.9,1.7,0.4]|0.0 |
[4.6,3.4,1.4,0.3]|0.0 |
[5.0,3.4,1.5,0.2]|0.0 |
[4.4,2.9,1.4,0.2]|0.0 |
[4.9,3.1,1.5,0.1]|0.0 |
[5.4,3.7,1.5,0.2]|0.0 |
[4.8,3.4,1.6,0.2]|0.0 |
[4.8,3.0,1.4,0.1]|0.0 |
[4.3,3.0,1.1,0.1]|0.0 |
[5.8,4.0,1.2,0.2]|0.0 |
[5.7,4.4,1.5,0.4]|0.0 |
[5.4,3.9,1.3,0.4]|0.0 |
[5.1,3.5,1.4,0.3]|0.0 |
[5.7,3.8,1.7,0.3]|0.0 |
[5.1,3.8,1.5,0.3]|0.0 |
+-----------------+--------+
only showing top 20 rows

In [0]:
# Dividindo os dados em treino e teste
irisTreino, irisTeste = iris_transformado.randomSplit([0.7,0.3])
print(irisTreino.count())
print(irisTeste.count())

112
38

In [0]:
# Criando objeto para geração do modelo de classificação
objeto_nb = NaiveBayes(labelCol="labelCol", featuresCol="featuresCol")
# criando modelo de classificação
modelo_nb = objeto_nb.fit(irisTreino)

# Realizando previsões com o modelo criado
previsao_nb = modelo_nb.transform(irisTeste)
previsao_nb.show(truncate=False)

+-----------------+--------+-------------------------------------------------------------+-------------------------------------------------------------+----------+
featuresCol |labelCol|rawPrediction |probability |prediction|
+-----------------+--------+-------------------------------------------------------------+-------------------------------------------------------------+----------+
[4.4,3.2,1.3,0.2]|0.0 |[-10.926671649997354,-12.287752064784536,-12.775752930397449]|[0.7073309753538999,0.18134808066808422,0.11132094397801581] |0.0 |
[4.6,3.2,1.4,0.2]|0.0 |[-11.261009731186006,-12.585471368960283,-13.079625271528535]|[0.7001838513334249,0.18621116166584387,0.11360498700073124] |0.0 |
[4.7,3.2,1.3,0.2]|0.0 |[-11.140194477313141,-12.55226802381891,-13.062534186660063] |[0.7194752322891497,0.1752911938150471,0.10523357389580318] |0.0 |
[4.8,3.0,1.4,0.1]|0.0 |[-10.825740854675463,-12.198232618016108,-12.706950776614962]|[0.7112981762663385,0.18029600175513802,0.10840582197852353] |0.0 |
[4.8,3.4,1.6,0.2]|0.0 |[-12.006089791470004,-13.330932125572698,-13.8458703408804] |[0.7019035574820167,0.1865974603106315,0.11149898220735174] |0.0 |
[4.9,2.4,3.3,1.0]|1.0 |[-17.11824365278141,-15.748386963895838,-15.822059727487323] |[0.11641297451173874,0.4580602303804036,0.4255267951078577] |1.0 |
[4.9,2.5,4.5,1.7]|2.0 |[-22.04354908347112,-19.028590220987862,-18.848340471358995] |[0.021832383815955834,0.4451241473937868,0.5330434687902573] |2.0 |
[4.9,3.0,1.4,0.2]|0.0 |[-11.255780109051237,-12.523621207044156,-13.016718636526381]|[0.68807874348756,0.1936518939399555,0.11826936257248453] |0.0 |
[5.0,3.4,1.5,0.2]|0.0 |[-11.956448813369068,-13.385900766776116,-13.924373008099465]|[0.7250649688882174,0.17360963636354862,0.10132539474823395] |0.0 |
[5.0,3.6,1.4,0.2]|0.0 |[-11.98321173317483,-13.590891556240452,-14.16137606240822] |[0.7612660782891071,0.15252112585361308,0.0862127958572798] |0.0 |
[5.2,3.5,1.5,0.2]|0.0 |[-12.208173589638204,-13.725427799940949,-14.290404457906924]|[0.7440669575786214,0.16318374196031132,0.09274930046106729] |0.0 |
[5.2,4.1,1.5,0.1]|0.0 |[-12.505565959386022,-14.467309560109197,-15.125294023877343]|[0.8241062977212367,0.11588008691628698,0.06001361536247633] |0.0 |
[5.4,3.0,4.5,1.5]|1.0 |[-22.228571628749464,-19.810932249721564,-19.772180744310837]|[0.041872562648943595,0.46978265993434243,0.4883447774167139]|2.0 |
[5.4,3.9,1.3,0.4]|0.0 |[-13.121777938001275,-14.786186556925715,-15.383946286347271]|[0.7731398615861788,0.14635736651456116,0.08050277189926001] |0.0 |
[5.5,3.5,1.3,0.2]|0.0 |[-12.037717357664404,-13.747193096002993,-14.351816040257518]|[0.7813623926985015,0.1413958374653434,0.07724176983615522] |0.0 |
[5.6,2.5,3.9,1.1]|1.0 |[-19.23664196438279,-17.494242520384876,-17.556343067292417] |[0.08279386059170209,0.47283824591564844,0.44436789349264944]|1.0 |
[5.8,2.7,5.1,1.9]|2.0 |[-24.772537149945492,-21.351189413324988,-21.16282936879543] |[0.014584510372433253,0.4464412303743045,0.5389742592532623] |2.0 |
[5.8,2.7,5.1,1.9]|2.0 |[-24.772537149945492,-21.351189413324988,-21.16282936879543] |[0.014584510372433253,0.4464412303743045,0.5389742592532623] |2.0 |
[6.0,2.2,5.0,1.5]|2.0 |[-22.740555133802786,-19.641376341419132,-19.510415876557055]|[0.020634337830189517,0.45766403432620517,0.5217016278436054]|2.0 |
[6.0,2.7,5.1,1.6]|1.0 |[-23.83829076567781,-20.815883577964804,-20.71149454949886] |[0.022552241190485187,0.46323831288834805,0.5142094459211667]|2.0 |
+-----------------+--------+-------------------------------------------------------------+-------------------------------------------------------------+----------+
only showing top 20 rows

In [0]:
# Avaliar a performance do modelo usando métrica de área sobre a curva, quanto mais próximo de 1 melhor
obj_avaliar_nb = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="labelCol", metricName="accuracy")
avaliar_nb = obj_avaliar_nb.evaluate(previsao_nb)
print(avaliar_nb)

0.8947368421052632